In [ ]:
import pandas as pd
import numpy as np
from pandas import read_csv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split

In [ ]:
#Load data
dataframe = pd.read_csv("GSE113865_series_matrix.txt", delimiter='\t')
dataset=dataframe.values
x1=dataset[:,1:7].astype(float)
x1 = x1.T
target=np.array([0,1,0,1,0,1])
y=np.array([0,1,0,1,0,1])

#Check the shape and range of the inputs/outputs to make sure everything matches
x1.shape
y.shape
x1.max()
x1.min()

In [ ]:
#Conduct data normalization
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x = scaler.fit_transform(x1)
x = x.astype('float32')
y = y.astype('float32')

In [ ]:
#Split data into training and test data sets
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.1, random_state=42)

In [ ]:
#Create the network model
#I used dropout and L1/L2 regulation to help avoid overfitting my data
model = Sequential()
model.add(Dense(64, input_dim=x.shape[1], activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(1 if len(np.unique(y)) == 2 else len(np.unique(y)), activation='sigmoid' if len(np.unique(y)) == 2 else 'softmax'))

#Compile the model    
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

#Run the training loop
history = model.fit(x_train, y_train, 
                    epochs=16, 
                    batch_size=50, 
                    validation_data=(x_test, y_test))

In [ ]:
#Evaluate the trained model
score = model.evaluate(x, y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])